In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Height"
cohort = "GSE102130"

# Input paths
in_trait_dir = "../../input/GEO/Height"
in_cohort_dir = "../../input/GEO/Height/GSE102130"

# Output paths
out_data_file = "../../output/preprocess/Height/GSE102130.csv"
out_gene_data_file = "../../output/preprocess/Height/gene_data/GSE102130.csv"
out_clinical_data_file = "../../output/preprocess/Height/clinical_data/GSE102130.csv"
json_path = "../../output/preprocess/Height/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Single cell RNA-seq analysis of K27M-mutant glioma"
!Series_summary	"To understand the diversity of expression states within K27M-mutant gliomas, we profiled 4058 single cells, primarily from 6 K27M-mutant gliomas, by single cell RNA-seq"
!Series_overall_design	"Tumors were disaggregated, sorted into single cells, and profiled by Smart-seq2."
!Series_overall_design	"------------------------------------------------------"
!Series_overall_design	"Authors state that  ""We are currently unable to deposit the raw data"
!Series_overall_design	"due to privacy concerns, but will have the raw data available once"
!Series_overall_design	"we obtain permission from our institution."""
!Series_overall_design	"Please contact the submitter directly regarding access to raw data on a case-by-case basis."
Sample Characteristics Dictionary:
{0: ['sample type: human tumor', 'sample type: patient-derived xenogrft (PDX)', 'sample type: patient-derived xenograft(PDX)', 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# This dataset is about K27M-mutant gliomas analyzed with single-cell RNA-seq
# It explicitly mentions profiling cells by RNA-seq, so gene expression data should be available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary, we don't see height/age/gender information
# The only key is 0, which contains sample type information, but not our variables of interest
trait_row = None  # Height data is not available
age_row = None    # Age data is not available 
gender_row = None # Gender data is not available

# 2.2 Data Type Conversion
# Even though we don't have the data, we'll define conversion functions in case they're needed elsewhere
def convert_trait(value):
    # For height, we would normally extract a numeric value in cm or inches
    if value is None or pd.isna(value):
        return None
    
    # Extract the value part after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        # Try to convert to float for continuous height measurement
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_age(value):
    # For age, we would normally extract a numeric value in years
    if value is None or pd.isna(value):
        return None
    
    # Extract the value part after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        # Try to convert to float for continuous age
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    # For gender, we would convert to binary: female=0, male=1
    if value is None or pd.isna(value):
        return None
    
    # Extract the value part after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if isinstance(value, str):
        value = value.lower()
        if 'female' in value or 'f' == value:
            return 0
        elif 'male' in value or 'm' == value:
            return 1
    
    return None

# 3. Save Metadata
# Initial filtering - trait data is not available (trait_row is None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, examine the structure of both files to understand their format
import gzip

# Check the size of the matrix file first
import os
matrix_file_size = os.path.getsize(matrix_file)
print(f"Matrix file size: {matrix_file_size} bytes")

# Peek at the matrix file structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    lines_found = 0
    data_marker_found = False
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            data_marker_found = True
            # Read the next line which should be the header
            next_line = next(file, None)
            if next_line:
                print(f"Next line after marker: {next_line.strip()}")
                lines_found += 1
            else:
                print("No data after marker")
            break
        if i >= 100:  # Limit search to first 100 lines
            break
    
    if not data_marker_found:
        print("Matrix table marker not found in first 100 lines")

# 3. Check the SOFT file as an alternative source of gene data
print("\nExamining SOFT file as alternative data source:")
with gzip.open(soft_file, 'rt') as file:
    for i, line in enumerate(file):
        if i < 20:  # Just show the first few lines to understand format
            print(line.strip())
        else:
            break

# 4. Attempt to extract gene expression data with error handling
try:
    print("\nAttempting to extract gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")

# 5. Set is_gene_available flag based on our findings
is_gene_available = False  # Based on our examination, this dataset doesn't contain usable gene expression data
print(f"\nGene expression data available: {is_gene_available}")

Matrix file size: 159095 bytes
Found data marker at line 70
Next line after marker: "ID_REF"	"GSM2724554"	"GSM2724555"	"GSM2724556"	"GSM2724557"	"GSM2724558"	"GSM2724559"	"GSM2724560"	"GSM2724561"	"GSM2724562"	"GSM2724563"	"GSM2724564"	"GSM2724565"	"GSM2724566"	"GSM2724567"	"GSM2724568"	"GSM2724569"	"GSM2724570"	"GSM2724571"	"GSM2724572"	"GSM2724573"	"GSM2724574"	"GSM2724575"	"GSM2724576"	"GSM2724577"	"GSM2724578"	"GSM2724579"	"GSM2724580"	"GSM2724581"	"GSM2724582"	"GSM2724583"	"GSM2724584"	"GSM2724585"	"GSM2724586"	"GSM2724587"	"GSM2724588"	"GSM2724589"	"GSM2724590"	"GSM2724591"	"GSM2724592"	"GSM2724593"	"GSM2724594"	"GSM2724595"	"GSM2724596"	"GSM2724597"	"GSM2724598"	"GSM2724599"	"GSM2724600"	"GSM2724601"	"GSM2724602"	"GSM2724603"	"GSM2724604"	"GSM2724605"	"GSM2724606"	"GSM2724607"	"GSM2724608"	"GSM2724609"	"GSM2724610"	"GSM2724611"	"GSM2724612"	"GSM2724613"	"GSM2724614"	"GSM2724615"	"GSM2724616"	"GSM2724617"	"GSM2724618"	"GSM2724619"	"GSM2724620"	"GSM2724621"	"GSM2724622"	"GSM272462

Extracted gene expression data is empty

Gene expression data available: False
